Analyzing the prediction of the training data

In [2]:
import matplotlib.pyplot as plt
import joblib

In [3]:
model = joblib.load('lasso_model.pkl')

In [4]:
y_train_pred = model.predict(X_train)

cm_test = confusion_matrix(y_train, y_train_pred)
tn, fp, fn, tp = cm_test.ravel()
print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"True Negatives (TN): {tn}")
print(f"False Negatives (FN): {fn}")

# Compute the AUC-ROC score
roc_auc = roc_auc_score(y_train, y_train_pred)

# Print the AUC-ROC score
print(f"AUC-ROC Score: {roc_auc:.4f}")


# Compute ROC curve
fpr, tpr, _ = roc_curve(y_train, y_train_pred)

# Plot the ROC Curve
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linestyle='-', label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Random classifier line
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve on Training Data")
plt.legend()
plt.show()

NameError: name 'X_train' is not defined

Analyzing the performace on the training data

In [ ]:
y_test_pred = model.predict(X_test)

cm_test = confusion_matrix(y_test, y_test_pred)
tn, fp, fn, tp = cm_test.ravel()
print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"True Negatives (TN): {tn}")
print(f"False Negatives (FN): {fn}")

# Compute the AUC-ROC score
roc_auc = roc_auc_score(y_test, y_test_pred)

# Print the AUC-ROC score
print(f"AUC-ROC Score: {roc_auc:.4f}")

# Compute ROC curve
fpr, tpr, _ = roc_curve(y_test, y_test_pred)

# Plot the ROC Curve
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, linestyle='-', label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Random classifier line
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC Curve on Testing Data")
plt.legend()
plt.show()

In [ ]:
# Extract the fitted steps
coefficients = model.named_steps['clf'].coef_.flatten()


# Pair each selected feature with its coefficient
feature_weights = list(zip(feature_names, coefficients))

# Sort features by absolute coefficient value (importance)
sorted_features = sorted(feature_weights, key=lambda x: abs(x[1]), reverse=True)

# Print top 20 most influential features
print("Top 20 features by absolute weight:")
for feature, weight in sorted_features[:20]:
    print(f"{feature}: {weight:.4f}")

non_zero_count = np.sum(coefficients != 0)
print(f"\nNumber of non-zero coefficients: {non_zero_count} out of {len(coefficients)}")